## 클래스별 파일 개수

In [10]:
import os
from collections import Counter
from pathlib import Path
import pandas as pd  # 테이블 보기 편함

root_dir = Path('./files/2_Animal_emotions')
jpg_counts = Counter()

# 1_Animal_Behavior 바로 아래 폴더만 대상
for subdir in root_dir.iterdir():
    if subdir.is_dir():
        jpg_count = len(list(subdir.rglob('*.jpg')))
        jpg_counts[subdir.name] = jpg_count

# 테이블 출력
df = pd.DataFrame.from_dict(jpg_counts, orient='index', columns=['JPG Count']).sort_values('JPG Count', ascending=False)
print(df)


               JPG Count
dog_happy          17355
dog_sad            14206
dog_anxious        11590
dog_relaxed         8699
dog_angry           8564
dog_confused        3286
cat_relaxed         2999
cat_happy           1221
cat_attentive        997
cat_sad              171


## train

In [ ]:
import os
import random
import shutil
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights
from PIL import Image
import librosa
import numpy as np
from collections import defaultdict, Counter

# =========================
# 0. 설정
# =========================
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

BEHAVIOR_ROOT = "files/1_Animal_Behavior"
EMOTION_ROOT = "files/2_Animal_emotions"
SOUND_ROOT = "files/3_Animal_Sound"
WORK_DIR = "files/work/omni_dataset"

# 🔥 샘플 제한 설정
MAX_SAMPLES_BEHAVIOR = 100000  # Behavior 총 샘플 수
MAX_SAMPLES_EMOTION = 100000    # Emotion 총 샘플 수
# Sound는 데이터가 적으므로 전체 사용

BATCH_SIZE = 16
EPOCHS = 30
LR_VIDEO = 1e-4
LR_AUDIO = 1e-5
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = 8
SR = 16000
MAX_AUDIO_LEN = SR * 5

LOSS_WEIGHTS = {
    "behavior": 1.0,
    "emotion": 0.8,
    "sound": 0.6
}

AUDIO_MODEL_NAME = "facebook/wav2vec2-base"
FEATURE_EXTRACTOR = Wav2Vec2FeatureExtractor.from_pretrained(AUDIO_MODEL_NAME)

print(f"🎯 Device: {DEVICE}")

# =========================
# 1. Dataset Preparation (개선)
# =========================
def collect_samples(root, exts):
    """모든 샘플 수집"""
    samples = []

    for class_dir in sorted(os.listdir(root)):
        class_path = os.path.join(root, class_dir)
        if not os.path.isdir(class_path):
            continue

        for root_dir, _, files in os.walk(class_path):
            for filename in files:
                if any(filename.lower().endswith(ext) for ext in exts):
                    file_path = os.path.join(root_dir, filename)
                    samples.append((class_dir, file_path))

    print(f"  → {len(samples)} samples, {len(set(s[0] for s in samples))} classes")
    return samples


def sample_balanced(samples, max_total_samples):
    """
    클래스별 균등 샘플링
    각 클래스당 max_samples_per_class = max_total_samples / num_classes
    """
    # 클래스별 그룹화
    class_samples = defaultdict(list)
    for label, path in samples:
        class_samples[label].append(path)
    
    num_classes = len(class_samples)
    max_per_class = max_total_samples // num_classes
    
    print(f"  🎯 Target: {max_total_samples} samples")
    print(f"  📊 {num_classes} classes → max {max_per_class} per class")
    
    # 클래스별 샘플링
    sampled = []
    for label, paths in class_samples.items():
        n_samples = min(len(paths), max_per_class)
        selected = random.sample(paths, n_samples)
        sampled.extend([(label, p) for p in selected])
        print(f"    {label}: {n_samples}/{len(paths)}")
    
    print(f"  ✅ Total sampled: {len(sampled)}")
    return sampled


def split_and_copy(samples, task_name):
    """8:1:1 split 후 복사"""
    random.shuffle(samples)
    class_samples = defaultdict(list)

    for label, path in samples:
        class_samples[label].append(path)

    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(WORK_DIR, split, task_name), exist_ok=True)

    for label, paths in class_samples.items():
        n = len(paths)
        n_train = int(n * 0.8)
        n_val = int(n * 0.1)

        splits = {
            "train": paths[:n_train],
            "val": paths[n_train:n_train+n_val],
            "test": paths[n_train+n_val:]
        }

        for split_name, split_paths in splits.items():
            for src in tqdm(split_paths, desc=f"{task_name}/{split_name}/{label}", leave=False):
                dst_dir = os.path.join(WORK_DIR, split_name, task_name, label)
                os.makedirs(dst_dir, exist_ok=True)

                dst_path = os.path.join(
                    dst_dir,
                    f"{label}_{os.path.basename(src)}"
                )
                shutil.copy(src, dst_path)


def prepare_dataset():
    """데이터셋 준비 (샘플 제한 적용)"""
    if os.path.exists(WORK_DIR):
        shutil.rmtree(WORK_DIR)

    print("\n📦 Collecting behavior...")
    behavior_all = collect_samples(BEHAVIOR_ROOT, ['.jpg', '.png', '.jpeg'])
    behavior = sample_balanced(behavior_all, MAX_SAMPLES_BEHAVIOR)

    print("\n📦 Collecting emotion...")
    emotion_all = collect_samples(EMOTION_ROOT, ['.jpg', '.png', '.jpeg'])
    emotion = sample_balanced(emotion_all, MAX_SAMPLES_EMOTION)

    print("\n📦 Collecting sound...")
    sound = collect_samples(SOUND_ROOT, ['.wav', '.mp3', '.m4a'])
    print("  ℹ️  Sound: Using all samples (no limit)")

    print("\n📋 Splitting & Copying...")
    split_and_copy(behavior, "behavior")
    split_and_copy(emotion, "emotion")
    split_and_copy(sound, "sound")

    print("\n✅ Dataset ready")

# =========================
# 2. Dataset Classes
# =========================
class ImageDataset(Dataset):
    def __init__(self, task_dir, augment=False):
        self.samples = []
        self.label_to_id = {}

        for label in sorted(os.listdir(task_dir)):
            label_dir = os.path.join(task_dir, label)
            if not os.path.isdir(label_dir):
                continue

            self.label_to_id[label] = len(self.label_to_id)

            for file in os.listdir(label_dir):
                if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                    self.samples.append((os.path.join(label_dir, file), label))

        print(f"  📊 {os.path.basename(task_dir)}: {len(self.samples)} samples, {len(self.label_to_id)} classes")

        if augment:
            self.transform = transforms.Compose([
                transforms.Resize((256,256)),
                transforms.RandomCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ColorJitter(0.2, 0.2, 0.2),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        img = self.transform(img)
        return img, self.label_to_id[label]


class AudioDataset(Dataset):
    def __init__(self, task_dir):
        self.samples = []
        self.label_to_id = {}

        for label in sorted(os.listdir(task_dir)):
            label_dir = os.path.join(task_dir, label)
            if not os.path.isdir(label_dir):
                continue

            self.label_to_id[label] = len(self.label_to_id)

            for file in os.listdir(label_dir):
                if file.lower().endswith(('.wav','.mp3','.m4a')):
                    self.samples.append((os.path.join(label_dir,file), label))

        print(f"  📊 {os.path.basename(task_dir)}: {len(self.samples)} samples, {len(self.label_to_id)} classes")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]

        try:
            waveform, _ = librosa.load(path, sr=SR, mono=True)
        except:
            waveform = np.zeros(MAX_AUDIO_LEN)

        if len(waveform) > MAX_AUDIO_LEN:
            waveform = waveform[:MAX_AUDIO_LEN]
        else:
            waveform = np.pad(waveform,(0,MAX_AUDIO_LEN-len(waveform)))

        inputs = FEATURE_EXTRACTOR(waveform, sampling_rate=SR, return_tensors="pt")
        return inputs.input_values.squeeze(0), self.label_to_id[label]

# =========================
# 3. Models
# =========================
class VideoMultiHead(nn.Module):
    def __init__(self, num_b, num_e):
        super().__init__()
        backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        self.behavior_head = nn.Linear(512, num_b)
        self.emotion_head = nn.Linear(512, num_e)

    def forward(self, x, task):
        feat = self.backbone(x).squeeze(-1).squeeze(-1)
        if task == "behavior":
            return self.behavior_head(feat)
        else:
            return self.emotion_head(feat)


class AudioModel(nn.Module):
    def __init__(self, num_classes, freeze_backbone=True):
        super().__init__()
        self.model = Wav2Vec2ForSequenceClassification.from_pretrained(
            AUDIO_MODEL_NAME,
            num_labels=num_classes,
            ignore_mismatched_sizes=True
        )

        if freeze_backbone:
            for param in self.model.wav2vec2.parameters():
                param.requires_grad = False

    def forward(self, x):
        return self.model(input_values=x).logits

# =========================
# 4. Training (진행 상황 추적 추가)
# =========================
def train():
    prepare_dataset()

    print("\n🔄 Loading datasets...")
    behavior_train = ImageDataset(os.path.join(WORK_DIR,"train","behavior"), augment=True)
    behavior_val = ImageDataset(os.path.join(WORK_DIR,"val","behavior"), augment=False)
    
    emotion_train = ImageDataset(os.path.join(WORK_DIR,"train","emotion"), augment=True)
    emotion_val = ImageDataset(os.path.join(WORK_DIR,"val","emotion"), augment=False)
    
    sound_train = AudioDataset(os.path.join(WORK_DIR,"train","sound"))
    sound_val = AudioDataset(os.path.join(WORK_DIR,"val","sound"))

    behavior_train_loader = DataLoader(behavior_train, BATCH_SIZE, True, num_workers=NUM_WORKERS, pin_memory=True)
    behavior_val_loader = DataLoader(behavior_val, BATCH_SIZE, False, num_workers=NUM_WORKERS//2, pin_memory=True)
    
    emotion_train_loader = DataLoader(emotion_train, BATCH_SIZE, True, num_workers=NUM_WORKERS, pin_memory=True)
    emotion_val_loader = DataLoader(emotion_val, BATCH_SIZE, False, num_workers=NUM_WORKERS//2, pin_memory=True)
    
    sound_train_loader = DataLoader(sound_train, BATCH_SIZE, True, num_workers=2, pin_memory=True)
    sound_val_loader = DataLoader(sound_val, BATCH_SIZE, False, num_workers=1, pin_memory=True)

    print(f"\n📦 DataLoaders:")
    print(f"  Behavior: {len(behavior_train_loader)} train batches, {len(behavior_val_loader)} val batches")
    print(f"  Emotion: {len(emotion_train_loader)} train batches, {len(emotion_val_loader)} val batches")
    print(f"  Sound: {len(sound_train_loader)} train batches, {len(sound_val_loader)} val batches")

    # 모델
    video_model = VideoMultiHead(
        len(behavior_train.label_to_id),
        len(emotion_train.label_to_id)
    ).to(DEVICE)

    audio_model = AudioModel(
        len(sound_train.label_to_id),
        # freeze_backbone=True
        freeze_backbone=False
    ).to(DEVICE)

    # Optimizer
    video_opt = torch.optim.AdamW(video_model.parameters(), lr=LR_VIDEO, weight_decay=0.01)
    audio_opt = torch.optim.AdamW(audio_model.parameters(), lr=LR_AUDIO, weight_decay=0.01)

    # Scaler
    video_scaler = torch.amp.GradScaler("cuda")
    audio_scaler = torch.amp.GradScaler("cuda")

    criterion = nn.CrossEntropyLoss()

    best_avg_acc = 0
    history = []

    for epoch in range(EPOCHS):
        print(f"\n{'='*60}")
        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"{'='*60}")
        
        video_model.train()
        audio_model.train()

        # Task별 loss 누적
        loss_b_total, loss_e_total, loss_s_total = 0, 0, 0

        # -------- 1. Behavior --------
        print(f"\n🐾 Training Behavior...")
        for imgs, labels in tqdm(behavior_train_loader, desc="Behavior", leave=False):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

            with torch.amp.autocast("cuda"):
                logits = video_model(imgs, "behavior")
                loss = criterion(logits, labels) * LOSS_WEIGHTS["behavior"]

            video_opt.zero_grad()
            video_scaler.scale(loss).backward()
            video_scaler.step(video_opt)
            video_scaler.update()

            loss_b_total += loss.item()

        avg_loss_b = loss_b_total / len(behavior_train_loader)
        print(f"  → Avg Loss: {avg_loss_b:.4f}")

        # -------- 2. Emotion --------
        print(f"\n😊 Training Emotion...")
        for imgs, labels in tqdm(emotion_train_loader, desc="Emotion", leave=False):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

            with torch.amp.autocast("cuda"):
                logits = video_model(imgs, "emotion")
                loss = criterion(logits, labels) * LOSS_WEIGHTS["emotion"]

            video_opt.zero_grad()
            video_scaler.scale(loss).backward()
            video_scaler.step(video_opt)
            video_scaler.update()

            loss_e_total += loss.item()

        avg_loss_e = loss_e_total / len(emotion_train_loader)
        print(f"  → Avg Loss: {avg_loss_e:.4f}")

        # -------- 3. Sound --------
        print(f"\n🔊 Training Sound...")
        for audios, labels in tqdm(sound_train_loader, desc="Sound", leave=False):
            audios, labels = audios.to(DEVICE), labels.to(DEVICE)

            with torch.amp.autocast("cuda"):
                logits = audio_model(audios)
                loss = criterion(logits, labels) * LOSS_WEIGHTS["sound"]

            audio_opt.zero_grad()
            audio_scaler.scale(loss).backward()
            audio_scaler.step(audio_opt)
            audio_scaler.update()

            loss_s_total += loss.item()

        avg_loss_s = loss_s_total / len(sound_train_loader)
        print(f"  → Avg Loss: {avg_loss_s:.4f}")

        # -------- Validation --------
        print(f"\n🔍 Validation...")
        video_model.eval()
        audio_model.eval()

        # Behavior Val
        correct_b, total_b = 0, 0
        with torch.no_grad():
            for imgs, labels in tqdm(behavior_val_loader, desc="Val Behavior", leave=False):
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                logits = video_model(imgs, "behavior")
                pred = logits.argmax(-1)
                correct_b += (pred == labels).sum().item()
                total_b += labels.size(0)
        acc_b = correct_b / total_b

        # Emotion Val
        correct_e, total_e = 0, 0
        with torch.no_grad():
            for imgs, labels in tqdm(emotion_val_loader, desc="Val Emotion", leave=False):
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                logits = video_model(imgs, "emotion")
                pred = logits.argmax(-1)
                correct_e += (pred == labels).sum().item()
                total_e += labels.size(0)
        acc_e = correct_e / total_e

        # Sound Val
        correct_s, total_s = 0, 0
        with torch.no_grad():
            for audios, labels in tqdm(sound_val_loader, desc="Val Sound", leave=False):
                audios, labels = audios.to(DEVICE), labels.to(DEVICE)
                logits = audio_model(audios)
                pred = logits.argmax(-1)
                correct_s += (pred == labels).sum().item()
                total_s += labels.size(0)
        acc_s = correct_s / total_s

        avg_acc = (acc_b + acc_e + acc_s) / 3

        print(f"\n📊 Results:")
        print(f"  Behavior: Loss {avg_loss_b:.4f} | Acc {acc_b:.4f} ({acc_b*100:.1f}%)")
        print(f"  Emotion:  Loss {avg_loss_e:.4f} | Acc {acc_e:.4f} ({acc_e*100:.1f}%)")
        print(f"  Sound:    Loss {avg_loss_s:.4f} | Acc {acc_s:.4f} ({acc_s*100:.1f}%)")
        print(f"  Average Acc: {avg_acc:.4f} ({avg_acc*100:.1f}%)")

        # History 저장
        history.append({
            'epoch': epoch+1,
            'loss_b': avg_loss_b,
            'loss_e': avg_loss_e,
            'loss_s': avg_loss_s,
            'acc_b': acc_b,
            'acc_e': acc_e,
            'acc_s': acc_s,
            'acc_avg': avg_acc
        })

        # Best 모델 저장
        if avg_acc > best_avg_acc:
            best_avg_acc = avg_acc
            torch.save({
                "video_model": video_model.state_dict(),
                "audio_model": audio_model.state_dict(),
                "behavior_label_to_id": behavior_train.label_to_id,
                "emotion_label_to_id": emotion_train.label_to_id,
                "sound_label_to_id": sound_train.label_to_id,
                "best_epoch": epoch+1,
                "best_acc": best_avg_acc,
                "history": history
            }, "pet_omni_best.pth")
            print(f"  💾 Saved new best model! (Acc: {best_avg_acc:.4f})")

    # 그래프
    print("\n📈 Generating training history plot...")
    plt.figure(figsize=(15, 5))

    plt.subplot(131)
    plt.plot([h['acc_b'] for h in history], 'b-', label='Behavior', linewidth=2)
    plt.xlabel('Epoch'); plt.ylabel('Accuracy')
    plt.title('Behavior Accuracy'); plt.ylim(0, 1); plt.grid(True, alpha=0.3); plt.legend()

    plt.subplot(132)
    plt.plot([h['acc_e'] for h in history], 'r-', label='Emotion', linewidth=2)
    plt.xlabel('Epoch'); plt.ylabel('Accuracy')
    plt.title('Emotion Accuracy'); plt.ylim(0, 1); plt.grid(True, alpha=0.3); plt.legend()

    plt.subplot(133)
    plt.plot([h['acc_s'] for h in history], 'g-', label='Sound', linewidth=2)
    plt.xlabel('Epoch'); plt.ylabel('Accuracy')
    plt.title('Sound Accuracy'); plt.ylim(0, 1); plt.grid(True, alpha=0.3); plt.legend()

    plt.tight_layout()
    plt.savefig('pet_omni_history.png', dpi=150, bbox_inches='tight')
    print("  ✅ Saved: pet_omni_history.png")

    print(f"\n🎉 Training Finished!")
    print(f"  Best Average Acc: {best_avg_acc:.4f} ({best_avg_acc*100:.1f}%)")


if __name__ == "__main__":
    train()

🎯 Device: cuda:1

📦 Collecting behavior...
  → 757113 samples, 25 classes
  🎯 Target: 100000 samples
  📊 25 classes → max 4000 per class
    CAT_ARCH: 2296/2296
    CAT_ARMSTRETCH: 4000/38483
    CAT_FOOTPUSH: 4000/9517
    CAT_GETDOWN: 4000/13421
    CAT_GROOMING: 4000/65029
    CAT_HEADING: 4000/11237
    CAT_LAYDOWN: 4000/21474
    CAT_LYING: 4000/12119
    CAT_ROLL: 4000/8513
    CAT_SITDOWN: 4000/18401
    CAT_TAILING: 4000/36960
    CAT_WALKRUN: 4000/30498
    DOG_BODYLOWER: 4000/79772
    DOG_BODYSCRATCH: 4000/15783
    DOG_BODYSHAKE: 4000/15296
    DOG_FEETUP: 4000/34365
    DOG_FOOTUP: 4000/52506
    DOG_HEADING: 4000/19052
    DOG_LYING: 4000/32129
    DOG_MOUNTING: 4000/5211
    DOG_SIT: 4000/79182
    DOG_TAILING: 4000/35824
    DOG_TAILLOW: 4000/8376
    DOG_TURN: 4000/21554
    DOG_WALKRUN: 4000/90115
  ✅ Total sampled: 98296

📦 Collecting emotion...
  → 69113 samples, 10 classes
  🎯 Target: 100000 samples
  📊 10 classes → max 10000 per class
    cat_attentive: 997/997
  


✅ Dataset ready

🔄 Loading datasets...
  📊 behavior: 11864 samples, 25 classes
  📊 behavior: 5301 samples, 25 classes
  📊 emotion: 44766 samples, 10 classes
  📊 emotion: 5592 samples, 10 classes
  📊 sound: 995 samples, 16 classes
  📊 sound: 121 samples, 16 classes

📦 DataLoaders:
  Behavior: 742 train batches, 332 val batches
  Emotion: 2798 train batches, 350 val batches
  Sound: 63 train batches, 8 val batches


Loading weights: 100%|██████████| 211/211 [00:00<00:00, 476.34it/s, Materializing param=wav2vec2.masked_spec_embed]                                            
Wav2Vec2ForSequenceClassification LOAD REPORT from: facebook/wav2vec2-base
Key                          | Status     | 
-----------------------------+------------+-
quantizer.weight_proj.weight | UNEXPECTED | 
quantizer.codevectors        | UNEXPECTED | 
project_hid.bias             | UNEXPECTED | 
project_q.weight             | UNEXPECTED | 
project_q.bias               | UNEXPECTED | 
quantizer.weight_proj.bias   | UNEXPECTED | 
project_hid.weight           | UNEXPECTED | 
projector.bias               | MISSING    | 
classifier.weight            | MISSING    | 
classifier.bias              | MISSING    | 
projector.weight             | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because miss


Epoch 1/30

🐾 Training Behavior...


  → Avg Loss: 2.0376

😊 Training Emotion...


...

  → Avg Loss: 0.4016

🔊 Training Sound...


  → Avg Loss: 0.6675

🔍 Validation...



📊 Results:
  Behavior: Loss 0.3067 | Acc 0.5127 (51.3%)
  Emotion:  Loss 0.4016 | Acc 0.7121 (71.2%)
  Sound:    Loss 0.6675 | Acc 0.7107 (71.1%)
  Average Acc: 0.6452 (64.5%)
  💾 Saved new best model! (Acc: 0.6452)

Epoch 10/30

🐾 Training Behavior...


  → Avg Loss: 0.2727

😊 Training Emotion...


Emotion:  18%|█▊        | 503/2798 [00:15<01:01, 37.17it/s]